## CNN sentence classifier - Hyperparameter tuning with reduced sentence length of 50 tokens.


In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 50


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 10 ms, sys: 14.8 ms, total: 24.8 ms
Wall time: 23.9 ms


## Tokenize the training data set and build a vocabulary from it


In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, tokenizer = tokenize_sentences(x_train, max_len)
x_val_ids, tokenizer   = tokenize_sentences(x_val, max_len)


Loading previously created Tokenizer
Loading previously created Tokenizer


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer)

Loading previously created Embedding Matrix
CPU times: user 1.51 ms, sys: 293 µs, total: 1.8 ms
Wall time: 1.56 ms


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen50.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16], [16,32], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal50",
                      model_dir        = "../models/CNNBal50",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.7135 - accuracy: 0.5826 - val_loss: 0.5903 - val_accuracy: 0.6980
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230148/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4713 - accuracy: 0.7812 - val_loss: 0.4163 - val_accuracy: 0.8142
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230148/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.3110 - accuracy: 0.8710 - val_loss: 0.3535 - val_accuracy: 0.8585
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230148/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1991 - accuracy: 0.9257 - val_loss: 0.2524 - val_accuracy: 0.9060
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230148/assets
Epoch 5/20
288/288 [=====

288/288 [==============================] - 1s 5ms/step - loss: 0.0902 - accuracy: 0.9697 - val_loss: 0.3703 - val_accuracy: 0.8879
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0662 - accuracy: 0.9801 - val_loss: 0.2281 - val_accuracy: 0.9299
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230259/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0388 - accuracy: 0.9883 - val_loss: 0.3045 - val_accuracy: 0.9227
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0354 - accuracy: 0.9882 - val_loss: 0.2098 - val_accuracy: 0.9485
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230259/assets
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0266 - accuracy: 0.9903 - val_loss: 0.3043 - val_accuracy: 0.9304
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0253 - accuracy: 0.9921 - val_loss: 0.2502 - val_accuracy: 0.9426
Epoch 11

288/288 [==============================] - 1s 5ms/step - loss: 0.0082 - accuracy: 0.9978 - val_loss: 0.5434 - val_accuracy: 0.9024
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0101 - accuracy: 0.9974 - val_loss: 0.5491 - val_accuracy: 0.9105
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0069 - accuracy: 0.9977 - val_loss: 0.3374 - val_accuracy: 0.9394
Epoch 16/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0066 - accuracy: 0.9985 - val_loss: 0.4354 - val_accuracy: 0.9236
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0073 - accuracy: 0.9983 - val_loss: 0.4765 - val_accuracy: 0.9250
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.4977 - val_accuracy: 0.9268
Epoch 19/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0091 - accuracy: 0.9966 - val_loss: 0.5483 - val_accuracy: 0.9222
Epoch 20/20

288/288 [==============================] - 1s 5ms/step - loss: 0.0069 - accuracy: 0.9974 - val_loss: 0.4422 - val_accuracy: 0.9308



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6416 - accuracy: 0.6294 - val_loss: 0.4018 - val_accuracy: 0.8264
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230549/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.3973 - accuracy: 0.8232 - val_loss: 0.4008 - val_accuracy: 0.8205
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2332 - accuracy: 0.9082 - val_loss: 0.2953 - val_accuracy: 0.8752
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230549/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1301 - accuracy: 0.9566 - val

288/288 [==============================] - 1s 5ms/step - loss: 0.0705 - accuracy: 0.9797 - val_loss: 0.2508 - val_accuracy: 0.9155
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230657/assets
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0472 - accuracy: 0.9844 - val_loss: 0.2620 - val_accuracy: 0.9222
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230657/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 0.1522 - val_accuracy: 0.9552
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-230657/assets
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0246 - accuracy: 0.9936 - val_loss: 0.2010 - val_accuracy: 0.9453
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0167 - accuracy: 0.9955 - val_loss: 0.3851 - val_accuracy: 0.9150
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0

...

288/288 [==============================] - 2s 6ms/step - loss: 0.0063 - accuracy: 0.9977 - val_loss: 0.4295 - val_accuracy: 0.9313
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0056 - accuracy: 0.9981 - val_loss: 0.4856 - val_accuracy: 0.9241



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 32
Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 16, Filter Count: 32
Adding Convolution: Kernel Size: 32, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.6147 - accuracy: 0.6604 - val_loss: 0.4729 - val_accuracy: 0.7821
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000523/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3663 - accuracy: 0.8399 - val_loss: 0.2796 - val_accuracy: 0.8906
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000523/asse

Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1827 - accuracy: 0.9299 - val_loss: 0.3178 - val_accuracy: 0.8865
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000646/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0826 - accuracy: 0.9721 - val_loss: 0.2228 - val_accuracy: 0.9250
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000646/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0457 - accuracy: 0.9850 - val_loss: 0.2048 - val_accuracy: 0.9340
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000646/assets
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0336 - accuracy: 0.9882 - val_loss: 0.2149 - val_accuracy: 0.9385
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-000646/assets
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0258 - accuracy: 0.9915 - val_loss: 0.2290 - val_accurac

Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0054 - accuracy: 0.9992 - val_loss: 0.3816 - val_accuracy: 0.9141
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0030 - accuracy: 0.9998 - val_loss: 0.2289 - val_accuracy: 0.9462
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0124 - accuracy: 0.9959 - val_loss: 0.4618 - val_accuracy: 0.8978
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0192 - accuracy: 0.9928 - val_loss: 0.3609 - val_accuracy: 0.9218
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0171 - accuracy: 0.9948 - val_loss: 0.3677 - val_accuracy: 0.9290
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0052 - accuracy: 0.9989 - val_loss: 0.3492 - val_accuracy: 0.9363
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 0.5564 - val_accuracy: 0.9118
Ep

Epoch 14/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0085 - accuracy: 0.9969 - val_loss: 0.4966 - val_accuracy: 0.9109
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0038 - accuracy: 0.9988 - val_loss: 0.7749 - val_accuracy: 0.8865
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0126 - accuracy: 0.9949 - val_loss: 0.2285 - val_accuracy: 0.9521
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0122 - accuracy: 0.9959 - val_loss: 0.2516 - val_accuracy: 0.9480
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.3639 - val_accuracy: 0.9326
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0044 - accuracy: 0.9987 - val_loss: 0.3029 - val_accuracy: 0.9390
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0023 - accuracy: 0.9990 - val_loss: 0.2615 - val_accuracy: 0.9620

Adding Convolution: Kernel Size: 4, Filter Count: 8
Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Adding Convolution: Kernel Size: 32, Filter Count: 64
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.6248 - accuracy: 0.6496 - val_loss: 0.5980 - val_accuracy: 0.7048
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-001148/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3770 - accuracy: 0.8336 - val_loss: 0.4617 - val_accuracy: 0.7957
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-001148/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.2007 - accuracy: 0.9198 - val_loss: 0.2145 - val_accuracy: 0.9227
INFO:tensorflow:Assets written to: ../models/CNNBal50/210410-001148/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0943 - accuracy: 0.9674 - val_loss: 0.3604 - val_accuracy: 0





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/